In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import os
import joblib
import gc
import warnings
import sys
warnings.filterwarnings('ignore')
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras import regularizers
import math
from sklearn.model_selection import train_test_split
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras import layers
from scipy.stats import skew
from tensorflow.keras.optimizers import Adam
from sklearn.tree import DecisionTreeClassifier
from joblib import parallel_backend
import dill
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from imblearn.over_sampling import SMOTE
import pickle
from sklearn.metrics import confusion_matrix
sb.set()

In [2]:
def label_prefog(dataset,window_length = 1):
  dataset.drop(index = list(dataset[dataset['Action'] == 0].index),inplace=True)
  window_length = 64*window_length

  fog_index=[]
  for i in dataset.index:
      if dataset.loc[i,'Action'] == 2:
        fog_index.append(i)
  fog_index



  start_indices=[]
  for i in fog_index:
    if (dataset.loc[i-1,'Action']!=dataset.loc[i,'Action']):
      start_indices.append(i)


  prefog=[]
  for start in start_indices:
    prefog_start = [x for x in range(start-window_length,start)]
    prefog.append(prefog_start)

  prefog = [item for sublist in prefog for item in sublist]

  for i in prefog:
       dataset.loc[i,'Action'] = 3
  dataset['Action'] = dataset['Action'] - 1
  return dataset

In [3]:
dataset = pd.DataFrame()

In [4]:
data_path=r"C:\Users\Admin\Desktop\Freezing_Gait_Research\Data_1"
path=r"C:\Users\Admin\Desktop\Labelled_Files"

people = []
for person in os.listdir(data_path):
    if '.txt' in person:
        people.append(person)
for window_length in range(1,5):
    for person in people:
        name = person.split('R')[0]
        print (name)
        file = data_path+"/"+person
        temp = pd.read_csv(file,delimiter= " ", header = None)
        print (person,' is read',end = '\t')
        if 2 in temp[max(temp.columns)].unique():
            print ('Adding {} to dataset'.format(person),end = '\t')
            temp.columns = ['time','A_F','A_V','A_L','L_F','L_V','L_L','T_F','T_V','T_L','Action']
            temp = label_prefog(temp,window_length).reset_index(drop=True)
            temp['name'] = name
            print ('{} is labelled'.format(person))
            dataset = pd.concat([dataset,temp],axis = 0)

        print ('')
    dataset.reset_index(drop =True,inplace=True)
    to_path = path + "/raw_labelled"
    to_name = to_path +"/win_"+str(window_length)+".csv"

    dataset.to_csv(to_name,index = False)


display(dataset)

S01
S01R01.txt  is read	Adding S01R01.txt to dataset	S01R01.txt is labelled

S01
S01R02.txt  is read	Adding S01R02.txt to dataset	S01R02.txt is labelled

S02
S02R01.txt  is read	Adding S02R01.txt to dataset	S02R01.txt is labelled

S02
S02R02.txt  is read	Adding S02R02.txt to dataset	S02R02.txt is labelled

S03
S03R01.txt  is read	Adding S03R01.txt to dataset	S03R01.txt is labelled

S03
S03R02.txt  is read	Adding S03R02.txt to dataset	S03R02.txt is labelled

S03
S03R03.txt  is read	
S04
S04R01.txt  is read	
S05
S05R01.txt  is read	Adding S05R01.txt to dataset	S05R01.txt is labelled

S05
S05R02.txt  is read	Adding S05R02.txt to dataset	S05R02.txt is labelled

S06
S06R01.txt  is read	Adding S06R01.txt to dataset	S06R01.txt is labelled

S06
S06R02.txt  is read	
S08
S08R01.txt  is read	Adding S08R01.txt to dataset	S08R01.txt is labelled

S09
S09R01.txt  is read	Adding S09R01.txt to dataset	S09R01.txt is labelled

S10
S10R01.txt  is read	
S01
S01R01.txt  is read	Adding S01R01.txt to dataset	

,time,A_F,A_V,A_L,L_F,L_V,L_L,T_F,T_V,T_L,Action,name
0,750000,-30,990,326,-45,972,181,-38,1000,29,0,S01
1,750015,-30,1000,356,-18,981,212,-48,1028,29,0,S01
2,750031,-20,990,336,18,981,222,-38,1038,9,0,S01
3,750046,-20,1000,316,36,990,222,-19,1038,9,0,S01
4,750062,0,990,316,36,990,212,-29,1038,29,0,S01
...,...,...,...,...,...,...,...,...,...,...,...,...
2886499,2549937,202,1019,188,-890,111,515,-155,971,291,0,S09
2886500,2549953,202,1009,178,-863,157,434,-155,980,281,0,S09
2886501,2549968,212,1000,158,-881,203,373,-155,971,281,0,S09
2886502,2549984,202,1019,148,-890,240,393,-155,961,252,0,S09


In [5]:
def create_window(act,window_length,dataframe):

  indices = list(dataframe[dataframe.Action == act].index)
  groups = []
  temp = []
  group_count = 0
  for i in range(len(indices)):
    if i == len(indices)-1:
      temp.append(indices[i])
      groups.append(temp)
      temp = []
      break
    temp.append(indices[i])
    if indices[i]+1 != indices[i+1]:
      group_count+=1
      groups.append(temp)
      temp = []

  fs = 64
  window_length = 1
  # window_length = window_length*fs

  final_dataframe = pd.DataFrame()
  for i in groups:
    required = math.floor(len(i)/(window_length*fs))

    req_index = i[0:(required*fs)]

    final_dataframe = pd.concat([final_dataframe,dataframe.iloc[req_index,:]],axis = 0)
  return final_dataframe

In [6]:
for window_length in range(1,5):

  # path = os.getcwd()+"/dataset_fog_release/dataset"
  name = path+"/raw_labelled/win_"+str(window_length)+".csv"
  dataframe = pd.read_csv(name)

  activities = []
  for act in range(3):
    activities.append(create_window(act,window_length,dataframe))
  to_write = pd.concat(activities,axis = 0)
  to_path = path + "/windows"+"/windowed_"+str(window_length)+".csv"
  to_write.to_csv(to_path,index = False)

In [ ]:
#window_length = 1
fs = 64
for window_length in range(1,5):
  w = window_length*fs
  FE_path = path + "/windows/windowed_"
  name = FE_path + str(window_length) + ".csv"
  dataframe = pd.read_csv(name)

  df = dataframe.drop(columns=['time','Action','name'])

  # Add jerk columns for each axis
  for axis in ["A_F", "A_V", "A_L", "L_F", "L_V", "L_L", "T_F", "T_V", "T_L"]:
    df[axis+"_jerk"] = df[axis].diff()
    
  acc_readings = ['A_F', 'A_V', 'A_L', 'L_F', 'L_V', 'L_L', 'T_F', 'T_V', 'T_L']
  df_acc = df[acc_readings]

  df['ankle_vector'] = df[['A_F', 'A_V', 'A_L']].values.tolist()
  df['trunk_vector'] = df[['T_F', 'T_V', 'T_L']].values.tolist()
  df['upper_leg_vector'] = df[['L_F', 'L_V', 'L_L']].values.tolist()

  df['ankle_vector'] = df['ankle_vector'].apply(np.array)
  df['trunk_vector'] = df['trunk_vector'].apply(np.array)
  df['upper_leg_vector'] = df['upper_leg_vector'].apply(np.array)

  df['angle_trunk_ankle'] = df.apply(lambda row: np.arccos(np.dot(row['trunk_vector'], row['ankle_vector']) / 
                                     (np.linalg.norm(row['trunk_vector']) * np.linalg.norm(row['ankle_vector']))), axis=1)

  df['angle_trunk_upper_leg'] = df.apply(lambda row: np.arccos(np.dot(row['trunk_vector'], row['upper_leg_vector']) / 
                                         (np.linalg.norm(row['trunk_vector']) * np.linalg.norm(row['upper_leg_vector']))), axis=1)

  df['angle_upper_leg_ankle'] = df.apply(lambda row: np.arccos(np.dot(row['upper_leg_vector'], row['ankle_vector']) / 
                                         (np.linalg.norm(row['upper_leg_vector']) * np.linalg.norm(row['ankle_vector']))), axis=1)

  rel_acc_trunk_ankle= df['trunk_vector'] - df['ankle_vector']
  rel_acc_trunk_upper_leg= df['trunk_vector'] - df['upper_leg_vector']
  rel_acc_upper_leg_ankle= df['upper_leg_vector'] - df['ankle_vector']

  df['rel_acc_trunk_ankle_x'] = rel_acc_trunk_ankle.apply(lambda v: v[0])
  df['rel_acc_trunk_ankle_y'] = rel_acc_trunk_ankle.apply(lambda v: v[1])
  df['rel_acc_trunk_ankle_z'] = rel_acc_trunk_ankle.apply(lambda v: v[2])
  df['rel_acc_trunk_upper_leg_x'] = rel_acc_trunk_upper_leg.apply(lambda v: v[0])
  df['rel_acc_trunk_upper_leg_y'] = rel_acc_trunk_upper_leg.apply(lambda v: v[1])
  df['rel_acc_trunk_upper_leg_z'] = rel_acc_trunk_upper_leg.apply(lambda v: v[2])
  df['rel_acc_upper_leg_ankle_x'] = rel_acc_upper_leg_ankle.apply(lambda v: v[0])
  df['rel_acc_upper_leg_ankle_y'] = rel_acc_upper_leg_ankle.apply(lambda v: v[1])
  df['rel_acc_upper_leg_ankle_z'] = rel_acc_upper_leg_ankle.apply(lambda v: v[2])

  df['rel_acc_trunk_ankle_mag'] = rel_acc_trunk_ankle.apply(np.linalg.norm)
  df['rel_acc_trunk_upper_leg_mag'] = rel_acc_trunk_upper_leg.apply(np.linalg.norm)
  df['rel_acc_upper_leg_ankle_mag'] = rel_acc_upper_leg_ankle.apply(np.linalg.norm)

  df['ankle_magnitude'] = df['ankle_vector'].apply(np.linalg.norm)
  df['trunk_magnitude'] = df['trunk_vector'].apply(np.linalg.norm)
  df['upper_leg_magnitude'] = df['upper_leg_vector'].apply(np.linalg.norm)

  df['correlation_trunk_ankle'] = df['trunk_magnitude'].corr(df['ankle_magnitude'])
  df['correlation_trunk_upper_leg'] = df['trunk_magnitude'].corr(df['upper_leg_magnitude'])
  df['correlation_upper_leg_ankle'] = df['upper_leg_magnitude'].corr(df['ankle_magnitude'])
    
  cross_correlation_df = pd.DataFrame()

  for i in range(0,len(df_acc),w):
      window_data = df_acc.iloc[i:i+w]
        
      # calculate cross-correlation
      for col1 in df_acc.columns:
          for col2 in df_acc.columns:
              if col1 != col2:  # avoid calculating correlation of a column with itself
                  corr = np.correlate(window_data[col1], window_data[col2])[0]  
                  cross_correlation_df.loc[i//w, 'cross_corr_'+col1+'_'+col2] = corr
                    
  df = pd.concat([df, cross_correlation_df], axis=1) 
    
  df=df.drop(columns=['ankle_vector', 'trunk_vector', 'upper_leg_vector'])

  stat = pd.DataFrame()

  col= list(df.columns)
  for s in col:
    print (s,end=" ")
    mn =[]
    mx =[]
    mi =[]
    var = []
    std = []
    mav = []
    rms =[]
    zcr =[]
    jerk_mn =[]  # Mean jerk
    p2p = []     # Peak-to-peak amplitude
    cf = []      # Crest factor
    sf = []      # Shape factor
    ifac = []    # Impulse factor
    skewness = []
    sma = []  # signal magnitude area
    wl = []   # waveform length
    cov=[]
    
    for i in range(0,len(df),w):
        window_data = df[s].iloc[i:i+w]
        mn_  = np.mean(window_data)
        mx_  = np.max(window_data)
        mi_  = np.min(window_data)
        var_  = np.var(window_data)
        std_  = np.std(window_data)
        mav_  = np.mean(abs(window_data))
        rms_  = np.sqrt(np.mean((window_data)**2))
        zcr_ = ((window_data > 0).astype(int)).diff().abs().sum() / 2.0
        jerk_mn_ = np.mean(window_data) if "jerk" in s else None  # Only compute mean jerk for jerk columns
        p2p_ = mx_ - mi_
        cf_ = mx_ / rms_ if rms_ != 0 else np.nan
        sf_ = rms_ / mav_ if mav_ != 0 else np.nan
        ifac_ = mx_ / mav_ if mav_ != 0 else np.nan
        skewness_ = skew(window_data)
        sma_ = np.sum(np.abs(window_data))
        wl_ = np.sum(np.abs(np.diff(window_data)))
        if i + w <= len(df):  # Ensure that the window doesn't extend beyond the data
            cov_ = np.cov(df.iloc[i:i+w].values.T)
            cov.append(cov_)
        else: 
            cov.append(None)
        
        mn.append(mn_)
        mx.append(mx_)
        mi.append(mi_)
        var.append(var_)
        std.append(std_)
        mav.append(mav_)
        rms.append(rms_)
        zcr.append(zcr_)
        jerk_mn.append(jerk_mn_)  # Append mean jerk
        p2p.append(p2p_)         # Append peak-to-peak
        cf.append(cf_)           # Append crest factor
        sf.append(sf_)           # Append shape factor
        ifac.append(ifac_)       # Append impulse factor
        skewness.append(skewness_)
        sma.append(sma_)
        wl.append(wl_)
        
    stat['skew_'+s] = skewness
    stat['sma_'+s] = sma
    stat['wl_'+s] = wl        
    stat['mean_'+s] = mn
    stat['max_'+s] = mx
    stat['min_'+s] = mi
    stat['var_'+s] = var
    stat['std_'+s] = std
    stat['rms_'+s] = rms
    stat['mav_'+s] = mav
    stat['zcr_'+s] = zcr
    stat['p2p_'+s] = p2p
    stat['cf_'+s] = cf
    stat['sf_'+s] = sf
    stat['ifac_'+s] = ifac
    stat['cov_'+s] = cov
    if "jerk" in s:
        stat['mean_jerk_'+s] = jerk_mn  # Add mean jerk column to stat dataframe

  import copy
  stat1 = copy.copy(stat)
  stat1['w'] = dataframe['Action'].iloc[[x for x in range(0,len(dataframe),w)]].to_list()
  order = ['w']
  order += stat1.columns.to_list()[:-1]
  stat1 = stat1[order]
  stat1.columns
  col = stat1.columns.to_list()
  col[0] = 0
  stat1.columns = col
  feature_name = path + "/features/time_"+str(window_length)+".csv"
  stat1.to_csv(feature_name, index = False)

A_F A_V A_L L_F L_V L_L T_F T_V T_L A_F_jerk A_V_jerk A_L_jerk L_F_jerk L_V_jerk L_L_jerk T_F_jerk T_V_jerk T_L_jerk angle_trunk_ankle angle_trunk_upper_leg angle_upper_leg_ankle rel_acc_trunk_ankle_x rel_acc_trunk_ankle_y rel_acc_trunk_ankle_z rel_acc_trunk_upper_leg_x rel_acc_trunk_upper_leg_y rel_acc_trunk_upper_leg_z rel_acc_upper_leg_ankle_x rel_acc_upper_leg_ankle_y rel_acc_upper_leg_ankle_z rel_acc_trunk_ankle_mag rel_acc_trunk_upper_leg_mag rel_acc_upper_leg_ankle_mag ankle_magnitude trunk_magnitude upper_leg_magnitude correlation_trunk_ankle correlation_trunk_upper_leg correlation_upper_leg_ankle cross_corr_A_F_A_V cross_corr_A_F_A_L cross_corr_A_F_L_F cross_corr_A_F_L_V cross_corr_A_F_L_L cross_corr_A_F_T_F cross_corr_A_F_T_V 

In [ ]:
from scipy.signal import butter, lfilter, welch

#window_length = 3
fs = 64
for window_length in range(1,5):
  w = window_length*fs
  FE_path = path + "/windows/windowed_"
  name = FE_path + str(window_length) + ".csv"
  dataframe = pd.read_csv(name)

  df = dataframe.drop(columns=['time','Action','name'])

  col= list(df.columns)

  order=5

  fi=pd.DataFrame()

  power = pd.DataFrame()
  bands = {'locomotor' :(0.5,3),'freeze' :(3,8)}
  
  # For dominant frequency and PSD
  dom_freq_df = pd.DataFrame()
  PSD_df = pd.DataFrame()

  for s in col:
      xtemp = []
      xtemp1 = []
      
      dom_freq_temp = []  # for dominant frequency
      PSD_temp = []  # for PSD
      
      for i in range(0,len(df),w):
          nyq=0.5*fs

          #locomotor band 0.5-3hz
          loc_low= 0.5/nyq
          loc_high=3/nyq

          #clipping off band from the window
          b, a = butter(order, [loc_low, loc_high], btype='band')
          y=lfilter(b,a,df[s].iloc[i:i+w])

          #total power in locomotor band
          e1=sum([x**2 for x in y])

          #freeze band 3-8hz
          frez_low= 3/nyq
          frez_high=8/nyq

          #clipping off band from the window
          b1, a1 = butter(order, [frez_low, frez_high], btype='band')
          y1=lfilter(b1,a1,df[s].iloc[i:i+w])
          #total power in locomotor band
          e2=sum([x**2 for x in y1])

          FI=e2/e1
          POW=e2+e1
          xtemp.append(FI)
          xtemp1.append(POW)
          
          # Compute PSD and dominant frequency
          freq, psd = welch(df[s].iloc[i:i+w], fs)
          dom_freq = freq[np.argmax(psd)]  # dominant frequency
          total_psd = np.sum(psd)  # total power spectral density
          
          dom_freq_temp.append(dom_freq)
          PSD_temp.append(total_psd)

      fi['FI'+s] = xtemp
      power['P'+s] = xtemp1
      
      dom_freq_df['dom_freq_'+s] = dom_freq_temp  # add to df
      PSD_df['PSD_'+s] = PSD_temp  # add to df

  print ("Freeze, power, dominant frequency, and PSD done")

  w = window_length*fs
  E=[]
  for i in range(0,len(df),w):
    energy = np.sum((df.iloc[i:i+w,:])**2)
    E.append(energy)
  E = pd.DataFrame(E)
  E.columns = ["EN_" + x for x in df.columns]

  #Entropy
  from scipy.signal import periodogram

  peak_f = pd.DataFrame()
  PSE = pd.DataFrame()
  for s in col:
    peakF = []
    pse = []
    for i in range(0,len(df),w):
        f,Pxx_den = periodogram(df[s].iloc[i:i+w],fs)
        p_norm = Pxx_den/sum(Pxx_den)
        p_norm = list(filter(lambda a: a != 0, p_norm))
        pse.append(-(np.sum(p_norm*np.log(p_norm))))
        peak = (fs/w)*max(Pxx_den)
        peakF.append(peak)
    PSE['ENt_'+s] = pse
    peak_f['peak_'+s] = peakF
  PSE.fillna(0,inplace = True)

  # Concatenating all the dataframes to create a final dataframe with all the features
  freq = pd.concat([fi,power,E,PSE,peak_f, dom_freq_df, PSD_df],axis = 1)

  feature_name = path + "/features/freq_"+str(window_length)+".csv"
  freq.to_csv(feature_name, index = False)
